In [1]:
%classpath add mvn org.apache.spark spark-sql_2.11 2.1.0
org.apache.log4j.Logger.getRootLogger().setLevel(org.apache.log4j.Level.ERROR);


null

In [2]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._ 


import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


In [3]:
val spark = SparkSession.builder() 
  .master("local[*]") 
  .config("spark.executor.memory", "3g")
  .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse")
  .appName("NohupReader")
  .getOrCreate()


org.apache.spark.sql.SparkSession@6b35b077

In [4]:
import spark.implicits._

val research_home: String = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/acald013/Research/")
val folder = s"${research_home}Scripts/Python/"
val prefix = "mu"

val nohup = spark.read.textFile(s"${folder}${prefix}.out")


org.apache.spark.sql.SparkSession$implicits$@4a56e54b

In [5]:
println(nohup.count)

6494


null

In [6]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Line(line: String, n: Long)
case class Run(runID: Long, date: String, method: String, cores: Int, epsilon: Double, mu: Int, delta: Int, methodTime: Double)
case class Stage(runID: Long, n: Long, timestamp: String, stage: String, stageTime: Double, load: Int, unit: String)
case class MDFrow(mdfID: Long, n: Long, method: String, epsilon: Double, mu: Int, delta: Int, stage: String, time: Double, load: Int, unit: String)

implicit class DatasetOps(ds: org.apache.spark.sql.Dataset[_]) {
    def display(rows: Int = 20) = {
        import com.twosigma.beakerx.scala.table.TableDisplay
        val columns = ds.columns
        val rowVals = ds.toDF.take(rows)
        val t = new TableDisplay(rowVals map (row => (columns zip row.toSeq).toMap))
        t.display()
    }
}

defined class Line
defined class Run
defined class Stage
defined class MDFrow
defined class DatasetOps


In [7]:
val lines = nohup.toDF("line").withColumn("n", monotonicallyIncreasingId).as[Line].cache()
val nLines = lines.count()
lines.show(10)

+--------------------+---+
|                line|  n|
+--------------------+---+
|FLOCKFINDER=Merge...|  0|
|WARNING:root:Sett...|  1|
|acald013@dblab-ra...|  2|
|acald013@dblab-ra...|  3|
|acald013@dblab-ra...|  4|
|acald013@dblab-ra...|  5|
|no org.apache.spa...|  6|
|starting org.apac...|  7|
|acald013@dblab-ra...|  8|
|acald013@dblab-ra...|  9|
+--------------------+---+
only showing top 10 rows



null

In [8]:
val indicesRun = lines.filter{ l => 
        l.line.contains("=== MergeLast Start ===") || l.line.contains("method=MergeLast,") ||
        l.line.contains("=== SpatialJoin Start ===") || l.line.contains("method=SpatialJoin,")
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexRun = spark.createDataset(indicesRun)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("runID","n")
    .cache
indexRun.show(10, truncate = false)

+-----+---+
|runID|n  |
+-----+---+
|0    |18 |
|0    |19 |
|0    |20 |
|0    |21 |
|0    |22 |
|0    |23 |
|0    |24 |
|0    |25 |
|0    |26 |
|0    |27 |
+-----+---+
only showing top 10 rows



null

In [9]:
val runs = indexRun.join(lines, "n").
    groupBy("runID").
    agg(max($"n").alias("n")).
    join(lines, "n").
    select("runID", "line").
    orderBy("runID").
    map{ row =>
        val runID = row.getInt(0)
        val line  = row.getString(1)
        var arr1  = line.split(" -> ")
        val date  = arr1(0)
        val arr2  = arr1(1).split(",")
        val method  = arr2(0).split("=")(1)
        val cores   = arr2(1).split("=")(1).toInt
        val epsilon = arr2(2).split("=")(1).toDouble
        val mu      = arr2(3).split("=")(1).toInt
        val delta   = arr2(4).split("=")(1).toInt
        val time    = arr2(5).split("=")(1).toDouble
        Run(runID, date, method, cores, epsilon, mu, delta, time)
    }.
    cache
val nRuns = runs.count()
runs.show(nRuns.toInt, truncate = false)

+-----+-----------------------+---------+-----+-------+---+-----+----------+
|runID|date                   |method   |cores|epsilon|mu |delta|methodTime|
+-----+-----------------------+---------+-----+-------+---+-----+----------+
|0    |2018-06-13 20:41:44,178|MergeLast|28   |100.0  |4  |6    |864.323   |
|1    |2018-06-13 20:54:47,264|MergeLast|28   |100.0  |5  |6    |764.412   |
|2    |2018-06-13 21:19:29,134|MergeLast|28   |100.0  |6  |6    |1459.496  |
|3    |2018-06-13 21:36:35,531|MergeLast|28   |100.0  |7  |6    |997.595   |
|4    |2018-06-13 21:54:11,679|MergeLast|28   |100.0  |8  |6    |1038.256  |
|5    |2018-06-13 22:12:35,102|MergeLast|28   |100.0  |8  |6    |1080.591  |
|6    |2018-06-13 22:30:11,021|MergeLast|28   |100.0  |7  |6    |1033.12   |
|7    |2018-06-13 22:55:51,993|MergeLast|28   |100.0  |6  |6    |1523.727  |
|8    |2018-06-13 23:09:18,158|MergeLast|28   |100.0  |5  |6    |782.743   |
|9    |2018-06-13 23:24:24,550|MergeLast|28   |100.0  |4  |6    |884.229   |

null

In [10]:
val sampleRuns = runs.filter($"method" === "MergeLast")
val nSampleRuns = sampleRuns.count()
sampleRuns.show(nSampleRuns.toInt, truncate = false)

+-----+-----------------------+---------+-----+-------+---+-----+----------+
|runID|date                   |method   |cores|epsilon|mu |delta|methodTime|
+-----+-----------------------+---------+-----+-------+---+-----+----------+
|0    |2018-06-13 20:41:44,178|MergeLast|28   |100.0  |4  |6    |864.323   |
|1    |2018-06-13 20:54:47,264|MergeLast|28   |100.0  |5  |6    |764.412   |
|2    |2018-06-13 21:19:29,134|MergeLast|28   |100.0  |6  |6    |1459.496  |
|3    |2018-06-13 21:36:35,531|MergeLast|28   |100.0  |7  |6    |997.595   |
|4    |2018-06-13 21:54:11,679|MergeLast|28   |100.0  |8  |6    |1038.256  |
|5    |2018-06-13 22:12:35,102|MergeLast|28   |100.0  |8  |6    |1080.591  |
|6    |2018-06-13 22:30:11,021|MergeLast|28   |100.0  |7  |6    |1033.12   |
|7    |2018-06-13 22:55:51,993|MergeLast|28   |100.0  |6  |6    |1523.727  |
|8    |2018-06-13 23:09:18,158|MergeLast|28   |100.0  |5  |6    |782.743   |
|9    |2018-06-13 23:24:24,550|MergeLast|28   |100.0  |4  |6    |884.229   |

null

In [11]:
val stages = lines.filter(_.line.contains("| ")).
    join(indexRun, "n").
    map{ m =>
        val n         = m.getLong(0)
        val line      = m.getString(1)
        val runID     = m.getInt(2)
        var arr1      = line.split(" -> ")
        val timestamp = arr1(0).trim
        val arr2      = arr1(1).split("\\|")
        val stage     = arr2(0).trim
        val time      = arr2(1).trim.dropRight(1).toDouble
        val arr3      = arr2(2).trim.split(" ")
        val load      = arr3(0).toInt
        val unit      = arr3(1)
        Stage(runID, n, timestamp, stage, time, load, unit)
    }.
    join(sampleRuns.select($"runID"), "runID").
    cache
    
println(stages.count())
stages.show(20, truncate = false)

1525
+-----+---+-----------------------+----------------------------------+---------+-----+----------+
|runID|n  |timestamp              |stage                             |stageTime|load |unit      |
+-----+---+-----------------------+----------------------------------+---------+-----+----------+
|0    |19 |2018-06-13 20:27:25,246|Reporting locations at t=0...     |5.19     |18093|points    |
|0    |35 |2018-06-13 20:28:12,137|1.Set of disks for t_i...         |46.89    |3329 |disks     |
|0    |36 |2018-06-13 20:28:17,720|Reporting locations at t=5...     |5.58     |18548|points    |
|0    |52 |2018-06-13 20:30:03,075|2.Set of disks for t_i+delta...   |105.36   |3603 |disks     |
|0    |53 |2018-06-13 20:30:24,235|3.Joining timestams               |21.16    |157  |candidates|
|0    |54 |2018-06-13 20:30:37,447|Indexing candidates...            |1.22     |157  |candidates|
|0    |55 |2018-06-13 20:30:37,672|Getting expansions...             |0.22     |1    |expansions|
|0    |56 |2018

null

In [12]:
val ml_stages = stages.join(runs, "runID").
    filter($"method" === "MergeLast").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "Reporting locations at t=\\d+", "0.Reporting locations")).
    withColumn("stage2", regexp_replace($"stage1", "Checking internal timestamps", "4.Checking internals")).
    withColumn("stage3", regexp_replace($"stage2", "\\.\\.\\.", "")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage3".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates")).
    filter(!$"stage".rlike("Indexing candidates")).
    filter(!$"stage".rlike("Getting expansions")).
    filter(!$"stage".rlike("Finding local maximals")).
    filter(!$"stage".rlike("Prunning duplicates and subsets"))
ml_stages.show(truncate = false)


+-----+---+---------+-------+---+-----+----------------------------+---------+
|runID|n  |method   |epsilon|mu |delta|stage                       |stageTime|
+-----+---+---------+-------+---+-----+----------------------------+---------+
|0    |19 |MergeLast|100.0  |4  |6    |0.Reporting locations       |5.19     |
|0    |35 |MergeLast|100.0  |4  |6    |1.Set of disks for t_i      |46.89    |
|0    |36 |MergeLast|100.0  |4  |6    |0.Reporting locations       |5.58     |
|0    |52 |MergeLast|100.0  |4  |6    |2.Set of disks for t_i+delta|105.36   |
|0    |53 |MergeLast|100.0  |4  |6    |3.Joining timestams         |21.16    |
|0    |58 |MergeLast|100.0  |4  |6    |4.Checking internals        |16.7     |
|0    |59 |MergeLast|100.0  |4  |6    |0.Reporting locations       |4.84     |
|0    |75 |MergeLast|100.0  |4  |6    |1.Set of disks for t_i      |27.68    |
|0    |76 |MergeLast|100.0  |4  |6    |0.Reporting locations       |4.89     |
|0    |92 |MergeLast|100.0  |4  |6    |2.Set of disk

null

In [13]:
val sj_stages = stages.join(runs, "runID").
    filter($"method" === "SpatialJoin").
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage".alias("stage0"), $"stageTime").
    withColumn("stage1", regexp_replace($"stage0", "\\.\\.\\.", "")).
    withColumn("stage2", regexp_replace($"stage1", "Reporting", "0.Reporting")).
    select($"runID", $"n", $"method", $"epsilon", $"mu", $"delta", $"stage2".alias("stage"), $"stageTime").
    filter(!$"stage".rlike("4.Distance Join phase")).
    filter(!$"stage".rlike("5.Getting candidates"))
sj_stages.show(truncate = false)



+-----+---+------+-------+---+-----+-----+---------+
|runID|n  |method|epsilon|mu |delta|stage|stageTime|
+-----+---+------+-------+---+-----+-----+---------+
+-----+---+------+-------+---+-----+-----+---------+



null

In [14]:
val data = runs.select($"runID", $"method", $"epsilon", $"mu", $"delta", $"methodTime".alias("time")).
    filter($"method" === "MergeLast").
    orderBy($"runID", $"epsilon", $"method").
    cache
println(data.count())
data.show(data.count().toInt, truncate = false)

25
+-----+---------+-------+---+-----+--------+
|runID|method   |epsilon|mu |delta|time    |
+-----+---------+-------+---+-----+--------+
|0    |MergeLast|100.0  |4  |6    |864.323 |
|1    |MergeLast|100.0  |5  |6    |764.412 |
|2    |MergeLast|100.0  |6  |6    |1459.496|
|3    |MergeLast|100.0  |7  |6    |997.595 |
|4    |MergeLast|100.0  |8  |6    |1038.256|
|5    |MergeLast|100.0  |8  |6    |1080.591|
|6    |MergeLast|100.0  |7  |6    |1033.12 |
|7    |MergeLast|100.0  |6  |6    |1523.727|
|8    |MergeLast|100.0  |5  |6    |782.743 |
|9    |MergeLast|100.0  |4  |6    |884.229 |
|10   |MergeLast|100.0  |4  |6    |862.928 |
|11   |MergeLast|100.0  |5  |6    |734.045 |
|12   |MergeLast|100.0  |6  |6    |1374.074|
|13   |MergeLast|100.0  |7  |6    |947.094 |
|14   |MergeLast|100.0  |8  |6    |980.409 |
|15   |MergeLast|100.0  |8  |6    |924.045 |
|16   |MergeLast|100.0  |7  |6    |1015.161|
|17   |MergeLast|100.0  |6  |6    |1562.109|
|18   |MergeLast|100.0  |5  |6    |761.712 |
|19   |

null

In [15]:
val d = data.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}ml_${prefix}.csv" ))
pw.write(s"$d\n")
pw.close

null

In [16]:
val d = ml_stages.union(sj_stages).collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}ml_${prefix}_stages.csv" ))
pw.write(s"$d\n")
pw.close

null

In [17]:
val mdf = lines.filter($"line".rlike("-> [A-K]\\."))
    .map{ m =>
        val line  = m.line.split(" -> ")(1)
        val arr   = line.split("\\[")
        val stage = arr(0).trim
        val time  = arr(1).trim.split("s")(0).toDouble
        val r = arr(2).trim.split(" ")(0)
        
        (stage, time, r, m.n)
    }
    .toDF("stage", "time", "load", "n")
    .join(indexRun, "n")
    .join(runs, "runID")
    .select("n", "runID", "stage", "epsilon", "mu", "delta", "time", "load")
mdf.show(false)

+---+-----+------------------------------------+-------+---+-----+------+-----+
|n  |runID|stage                               |epsilon|mu |delta|time  |load |
+---+-----+------------------------------------+-------+---+-----+------+-----+
|21 |0    |A.Indexing points...                |100.0  |4  |6    |8.325 |18093|
|22 |0    |B.Getting pairs...                  |100.0  |4  |6    |7.847 |33957|
|23 |0    |C.Computing centers...              |100.0  |4  |6    |2.636 |67914|
|24 |0    |D.Indexing centers...               |100.0  |4  |6    |1.849 |67914|
|25 |0    |E.Getting disks...                  |100.0  |4  |6    |6.876 |67914|
|26 |0    |F.Filtering less-than-mu disks...   |100.0  |4  |6    |0.84  |32499|
|27 |0    |G.Prunning duplicate candidates...  |100.0  |4  |6    |4.04  |19046|
|28 |0    |H.Indexing candidates...            |100.0  |4  |6    |4.606 |19046|
|29 |0    |I.Getting expansions...             |100.0  |4  |6    |0.331 |38041|
|30 |0    |J.Finding maximal disks...   

null

In [18]:
val indicesMdf = lines.filter{ l => 
        l.line.contains(" -> Setting mu=") || l.line.contains(" ->   berlin0-10,") 
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val indexMdf = spark.createDataset(indicesMdf)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("mdfID","n")
    .cache
indexMdf.show(15, truncate = false)

+-----+---+
|mdfID|n  |
+-----+---+
|0    |20 |
|0    |21 |
|0    |22 |
|0    |23 |
|0    |24 |
|0    |25 |
|0    |26 |
|0    |27 |
|0    |28 |
|0    |29 |
|0    |30 |
|0    |31 |
|0    |32 |
|0    |33 |
|1    |37 |
+-----+---+
only showing top 15 rows



null

In [19]:
val mdfInfo = indexMdf.groupBy($"mdfID").agg(max($"n").alias("n")).orderBy($"n").
    join(lines, "n").
    withColumn("timestamp", substring($"line", 122, 124)).
    select($"mdfID", $"timestamp")
mdfInfo.show(truncate = false)

+-----+---------+
|mdfID|timestamp|
+-----+---------+
|0    |  0      |
|1    |  5      |
|2    |  1      |
|3    |  6      |
|4    |  2      |
|5    |  7      |
|6    |  3      |
|7    |  8      |
|8    |  4      |
|9    |  9      |
|10   | 10      |
|11   |  0      |
|12   |  5      |
|13   |  1      |
|14   |  6      |
|15   |  2      |
|16   |  7      |
|17   |  3      |
|18   |  8      |
|19   |  4      |
+-----+---------+
only showing top 20 rows



null

In [20]:
val mdf2 = mdf.join(indexMdf, "n")
    .join(mdfInfo, "mdfID")
    .orderBy("n")

mdf2.show(false)

+-----+---+-----+------------------------------------+-------+---+-----+------+-----+---------+
|mdfID|n  |runID|stage                               |epsilon|mu |delta|time  |load |timestamp|
+-----+---+-----+------------------------------------+-------+---+-----+------+-----+---------+
|0    |21 |0    |A.Indexing points...                |100.0  |4  |6    |8.325 |18093|  0      |
|0    |22 |0    |B.Getting pairs...                  |100.0  |4  |6    |7.847 |33957|  0      |
|0    |23 |0    |C.Computing centers...              |100.0  |4  |6    |2.636 |67914|  0      |
|0    |24 |0    |D.Indexing centers...               |100.0  |4  |6    |1.849 |67914|  0      |
|0    |25 |0    |E.Getting disks...                  |100.0  |4  |6    |6.876 |67914|  0      |
|0    |26 |0    |F.Filtering less-than-mu disks...   |100.0  |4  |6    |0.84  |32499|  0      |
|0    |27 |0    |G.Prunning duplicate candidates...  |100.0  |4  |6    |4.04  |19046|  0      |
|0    |28 |0    |H.Indexing candidates..

null

In [21]:
val d = mdf2.collect.map(_.mkString(";")).mkString("\n")

import java.io._
val pw = new PrintWriter(new File(s"${folder}ml_${prefix}_mdf.csv" ))
pw.write(s"$d\n")
pw.close

null